# Riepilogo nazione

In [8]:
import altair as alt
from altair import datum
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import HTML, Markdown as md
import datetime as dt

from src.data.read_dataset import get_processed_dataset
from src.visualization import visualize as viz

df = get_processed_dataset('dpc_nazionale')

today = df.iloc[-1]
yesterday = df.iloc[-2]

In [2]:
html = '<p>Data aggiornamento: ' + str(today['data']) + '</p>'
html += viz.tabella_riassuntiva(today, yesterday)
HTML(html)

,Oggi,Variazione
Totale Casi,465726,16078
Totale Positivi,169302,13860
Tamponi,14132421,170392
Casi Testati,8564608.0,104872
Deceduti,36968,136
Dimessi Guariti,259456,2082
Terapia Intensiva,992,66
Incremento,9.50%,-0.28
Percentuale Positivi,5.44%,0.12
Percentuale Nuovi Positivi,15.33%,1.06


In [6]:
resize = alt.selection_interval(bind='scales')

def rolling_avg(
    field,
    title,
    data=df,
    tooltip=["data", "nuovi_positivi", "nuovi_deceduti", 'nuovi_tamponi', 'percentuale_nuovi_positivi'],
):
    base = (
        alt.Chart()
        .transform_window(averaged=f"mean({field})", frame=[-4, 3])
        .encode(x="data:T", tooltip=tooltip)
        .properties(height=200, width=600,)
    )

    c1 = base.mark_line().encode(y=alt.Y("averaged:Q", title=""),)

    c2 = base.mark_bar(opacity=0.1).encode(
        y=alt.Y(f"{field}:Q", title=""),
    )

    return (
        alt.layer(c1, c2, data=data)
        .properties(title={"text": [title], "subtitle": ["Dato mediato su 7 giorni"]})
        .add_selection(
            resize
        )
    )

In [7]:
c1 = rolling_avg('nuovi_positivi', title="Nuovi positivi") 
c2 = rolling_avg('nuovi_deceduti', title="Nuovi deceduti")
c3 = rolling_avg('nuovi_tamponi', title="Nuovi tamponi")
c4 = rolling_avg('percentuale_nuovi_positivi', title="Percentuali nuovi positivi")
alt.vconcat(c1, c2, c3, c4)

alt.VConcatChart(...)

In [22]:
def plot_line(field, title):
    base = alt.Chart(df).mark_line().encode(
        x=alt.X("data"),
        y=alt.Y(field, title="")
    ).properties(height=200, width=600,)
    
    return (
        base
        .properties(title={"text": [title]})
        .add_selection(
            resize
        )
    )

c1 = plot_line("incidenza", "Incidenza media")
c2 = plot_line("letalita", "Letalita")
c3 = plot_line("terapia_intensiva", "Terapia Intensiva")
c4 = plot_line("totale_positivi", "Totale positivi")
c5 = plot_line("percentuale_positivi", "Percentuale positivi")
alt.vconcat(c1, c3, c4, c5, c2)

alt.VConcatChart(...)